<a href="https://colab.research.google.com/github/samina-if/GenAi/blob/main/Challenge_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai gradio huggingface_hub requests

In [ ]:
!pip install transformers

In [11]:
!pip install pillow
!pip install diffusers

In [3]:
NEWS_API_KEY = "51b334d0221c4e4184fdb08ccb85c1fd"



In [4]:
import requests

def fetch_news(query="latest news", num_articles=5):
    url = f"https://newsapi.org/v2/everything?q={query}&apiKey={NEWS_API_KEY}"
    response = requests.get(url).json()
    articles = response.get('articles', [])[:num_articles]
    return [{"title": article['title'], "content": article['content']} for article in articles]


In [5]:
from transformers import pipeline

# Load a pre-trained summarization model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_text_local(text, tone="formal"):
    summary = summarizer(text, max_length=150, min_length=40, do_sample=False)
    return summary[0]['summary_text']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [6]:
def generate_social_media_post(summary, platform="LinkedIn"):
    if platform == "LinkedIn":
        return f"🌟 Trending News Update 🌟\n\n{summary}\n\n#LinkedIn #Trending"
    elif platform == "Facebook":
        return f"📢 Breaking News 📢\n\n{summary}\n\nStay tuned for more updates! #Facebook #News"


In [7]:
from PIL import Image, ImageDraw, ImageFont

def create_meme(text, image_path="default_meme.jpg"):
    try:
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 20)  # Use available fonts

        # Split text into lines to fit the meme
        lines = text.split(' ')
        meme_text = ' '.join(lines[:10]) + '\n' + ' '.join(lines[10:20])

        # Add text to image
        text_position = (10, image.height - 60)
        draw.multiline_text(text_position, meme_text, fill="white", font=font, align="center")

        # Save and return the meme
        output_path = "generated_meme.jpg"
        image.save(output_path)
        return output_path
    except Exception as e:
        return f"Error generating meme: {str(e)}"



In [8]:
from diffusers import StableDiffusionPipeline

# Load Stable Diffusion model
def generate_image_from_text(prompt):
    model_id = "runwayml/stable-diffusion-v1-5"  # Example Stable Diffusion model
    pipe = StableDiffusionPipeline.from_pretrained(model_id)
    pipe = pipe.to("cuda")  # If Colab GPU is available
    image = pipe(prompt).images[0]
    image.save("generated_image.png")
    return "generated_image.png"


In [12]:
import gradio as gr

In [21]:
#Working of Code if User select LinkedIn and image form generation against Prompt
import gradio as gr

# AI Agent Functionality
def ai_agent(prompt, platform, format_type):
    # Fetch relevant news articles
    news = fetch_news(prompt)
    summaries = [summarize_text_local(article['content']) for article in news]
    social_post = generate_social_media_post(summaries[0], platform)  # Use the first summary

    if format_type == "Text":
        return social_post  # Only return the text

    elif format_type == "Image":
        image_path = generate_image_from_text(summaries[0])  # Generate image
        return image_path  # Only return the image path

    elif format_type == "Meme":
        meme_path = create_meme(social_post, image_path="default_meme.jpg")  # Generate meme
        return meme_path  # Only return the meme path

# Gradio Wrapper
def gradio_wrapper(prompt, platform, format_type):
    result = ai_agent(prompt, platform, format_type)

    # Depending on the selected format, return appropriate output
    if format_type == "Text":
        return result, None  # Text output, no image
    else:
        return None, result  # No text output, only image

# Define Gradio Interface
with gr.Blocks(css=".gr-block { background-color: #f0f4ff; border-radius: 10px; padding: 10px; }") as iface:
    gr.Markdown(
        """
        <div style="text-align: center; background-color: #283593; color: white; padding: 15px; border-radius: 10px;">
            <h1 style="color: white;">🌟 Daily News AI Agent 🌟</h1>
            <p style="color: white;">Fetch, summarize, and create engaging content in text, image, or meme formats!</p>
        </div>
        """
    )


    with gr.Row():
        with gr.Column():
            prompt_input = gr.Textbox(label="🔍 News Topic or Prompt", placeholder="Type your topic here...", elem_id="prompt-box")
            platform_input = gr.Dropdown(choices=["LinkedIn", "Facebook"], label="📢 Platform", elem_id="platform-box")
            format_type_input = gr.Radio(choices=["Text", "Image", "Meme"], label="🖼️ Content Format", elem_id="format-type-box")

        with gr.Column():
            text_output = gr.Textbox(label="📝 Generated Text", interactive=False, elem_id="text-output")
            image_output = gr.Image(label="📷 Generated Image", interactive=False, elem_id="image-output")

    submit_btn = gr.Button("🚀 Submit", elem_id="submit-button")

    # Define functionality on button click
    submit_btn.click(
        fn=gradio_wrapper,
        inputs=[prompt_input, platform_input, format_type_input],
        outputs=[text_output, image_output]
    )

# Launch the interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d5edc05fae56d052e4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [22]:
#Working of Code if User select LinkedIn and post in the form of text  generation against Prompt
import gradio as gr

# AI Agent Functionality
def ai_agent(prompt, platform, format_type):
    # Fetch relevant news articles
    news = fetch_news(prompt)
    summaries = [summarize_text_local(article['content']) for article in news]
    social_post = generate_social_media_post(summaries[0], platform)  # Use the first summary

    if format_type == "Text":
        return social_post  # Only return the text

    elif format_type == "Image":
        image_path = generate_image_from_text(summaries[0])  # Generate image
        return image_path  # Only return the image path

    elif format_type == "Meme":
        meme_path = create_meme(social_post, image_path="default_meme.jpg")  # Generate meme
        return meme_path  # Only return the meme path

# Gradio Wrapper
def gradio_wrapper(prompt, platform, format_type):
    result = ai_agent(prompt, platform, format_type)

    # Depending on the selected format, return appropriate output
    if format_type == "Text":
        return result, None  # Text output, no image
    else:
        return None, result  # No text output, only image

# Define Gradio Interface
with gr.Blocks(css=".gr-block { background-color: #f0f4ff; border-radius: 10px; padding: 10px; }") as iface:
    gr.Markdown(
        """
        <div style="text-align: center; background-color: #283593; color: white; padding: 15px; border-radius: 10px;">
            <h1 style="color: white;">🌟 Daily News AI Agent 🌟</h1>
            <p style="color: white;">Fetch, summarize, and create engaging content in text, image, or meme formats!</p>
        </div>
        """
    )


    with gr.Row():
        with gr.Column():
            prompt_input = gr.Textbox(label="🔍 News Topic or Prompt", placeholder="Type your topic here...", elem_id="prompt-box")
            platform_input = gr.Dropdown(choices=["LinkedIn", "Facebook"], label="📢 Platform", elem_id="platform-box")
            format_type_input = gr.Radio(choices=["Text", "Image", "Meme"], label="🖼️ Content Format", elem_id="format-type-box")

        with gr.Column():
            text_output = gr.Textbox(label="📝 Generated Text", interactive=False, elem_id="text-output")
            image_output = gr.Image(label="📷 Generated Image", interactive=False, elem_id="image-output")

    submit_btn = gr.Button("🚀 Submit", elem_id="submit-button")

    # Define functionality on button click
    submit_btn.click(
        fn=gradio_wrapper,
        inputs=[prompt_input, platform_input, format_type_input],
        outputs=[text_output, image_output]
    )

# Launch the interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8a6856980fd63d4456.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
